## Code to work with ATM data along the flowline of Zachariae Isstrom

**by Jukes Liu**

**June 2019 ICESat-2 Hackweek**


In [235]:
#IMPORT PACKAGES
import os
import glob
import pandas as pd
import csv
import numpy as np
import math
import matplotlib.pyplot as plt
import pyproj
import geopandas as gpd
from shapely.geometry import Point, Polygon
from ipyleaflet import Map, Marker, basemaps #'OpenTopoMap'

## Read in ATM elevation CSV files into dataframes and each variable into 1D arrays:

In [238]:
#get all the relevant csv files as a list and sort in order of the flight path
#filelist = glob.glob("../data/ILATM2_2018*.csv")
filelist = glob.glob("../data_raw/ATM_2018/*.csv")
filelist.sort(reverse=True)
#test with first file in the filelist
# filelist = [filelist[0]] 

#create an empty DataFrame to hold the stitched together ATM data along the profile
df_total = pd.DataFrame()

#for each csv file
for file in filelist:
    #print(file)
    #read the csv file into a pandas DataFrame
    csv_df = pd.read_csv(file, skiprows=9)
    
    #concatenate (join) it to the df_total
    df_total = pd.concat([csv_df, df_total])

#check that we have all the data properly stitched together:
#print(df_total)

#Read variables from csv DataFrame into 1D arrays
ATM_lat = df_total.loc[:, df_total.keys()[1]].values
ATM_long = df_total.loc[:, df_total.keys()[2]].values
ATM_elev = df_total.loc[:, df_total.keys()[3]].values
TrackID = df_total.loc[:, df_total.keys()[-1]].values
NS_slope = df_total.loc[:,df_total.keys()[4]].values
EW_slope = df_total.loc[:,df_total.keys()[5]].values

#Set in new DataFrame called ATM_data
ATM_data = pd.DataFrame([ATM_lat, ATM_long, ATM_elev, TrackID, NS_slope, EW_slope])
headers = ['lat', "lon", "elev", "TrackID", "NS_slope", "EW_slope"]
ATM_data = ATM_data.transpose()
ATM_data.columns = headers
#ATM_data.head()

#Select only the data with TrackID 0:
ATM_track0 = ATM_data[ATM_data['TrackID'] == 0.0].copy()

#get the variables in arrays:
track0_lat = ATM_track0.lat.values
track0_long = ATM_track0.lon.values
track0_elev = ATM_track0.elev.values
track0_slope_NS = ATM_track0.NS_slope.values
track0_slope_EW = ATM_track0.EW_slope.values

ATM_track0.head()

,lat,lon,elev,TrackID,NS_slope,EW_slope
3,79.212155,334.848567,891.6397,0.0,0.002528,-0.003607
7,79.212039,334.850146,891.4861,0.0,0.003011,-0.003841
11,79.211923,334.851724,891.3228,0.0,0.003946,-0.003423
15,79.211808,334.853304,891.1725,0.0,0.004970,-0.002393
19,79.211692,334.854884,891.0138,0.0,0.005104,-0.003135


## Reproject lat and long into Greenland Polar Stereo coordinates (ESPG: 3413)

In [239]:
#Coordinate transformation function written by Fernando Paolo:
def transform_coord(proj1, proj2, x, y):
    """
    Transform coordinates from proj1 to proj2 (EPSG num).

    Example EPSG projs:
        Geodetic (lon/lat): 4326
        Polar Stereo AnIS (x/y): 3031
        Polar Stereo GrIS (x/y): 3413
    """
    # Set full EPSG projection strings
    proj1 = pyproj.Proj("+init=EPSG:"+str(proj1))
    proj2 = pyproj.Proj("+init=EPSG:"+str(proj2))
    return pyproj.transform(proj1, proj2, x, y)  # convert

#use the function to transform into Greenland Polar Stereo (PS_x, PS_y)
PS_x, PS_y = transform_coord(4326, 3413, track0_long, track0_lat)
print(PS_x)
print(PS_y)

[397916.20306121 397950.88495727 397985.54796972 ... 521498.057707
 521526.14084298 521554.12880888]
[-1102322.11492441 -1102323.06892135 -1102324.02879225 ...
 -1084654.329196   -1084639.2497822  -1084623.97262617]


## Calculate distance along the flowline:

In [241]:
#create an empty array to contain the distance along flow 
dist_along = np.empty(len(track0_lat), dtype=float)

#for each lat-long point:
for i in range(len(track0_lat)):
    #for the first point, set distance along flow to 0 m
    if i == 0:
        dist_along[i] = 0.0
    
    #for all other subsequent points:
    else:
        #calculate distance from the last point in meters
        x_diff = PS_x[i] - PS_x[i-1]
        y_diff = PS_y[i] - PS_y[i-1]
                
        #calculate distance from the previous point using the distance formula:    
        dist_last = math.sqrt(((x_diff)**2)+((y_diff)**2))
        #print(dist_last)
        
        #add it to the previous distance along the track to get the cumulative distance
        dist_along[i] = dist_last + dist_along[i-1]

    #check the array of distance along the track in meters:
    #print(dist_along[i], ATM_elev[i])

## Final data extracted into arrays
+ ATM_long: longitude of elev points
+ ATM_lat: latitude of elev points
+ PS_x: x-coords of elev points in Greenland Polar Stereo
+ PS_y: y-coords of elev points in Greenland Polar Stereo
+ ATM_elev: elevation points with z in meters
+ dist_along: distance along the flowline in meters

## ...and recombined into a dataframe called final_data:

In [243]:
variables = [track0_lat, track0_long, PS_x, PS_y, track0_elev, dist_along, track0_slope_NS, track0_slope_EW]
indices = ['ATM_lat', "ATM_long", "PS_x", "PS_y", "ATM_elev", "dist_along", 'slope_NS', 'slope_EW']

#create DataFrame from these variables
final_data = pd.DataFrame(variables)
final_data = final_data.transpose()
final_data.columns = indices

#SAVE TO CSV
final_data.to_csv('/home/jovyan/xtrak/data_prod/ATMprof_2018_slopes.csv',index=False)
final_data.head()

,ATM_lat,ATM_long,PS_x,PS_y,ATM_elev,dist_along,slope_NS,slope_EW
0,79.212155,334.848567,397916.203061,-1.102322e+06,891.6397,0.000000,0.002528,-0.003607
1,79.212039,334.850146,397950.884957,-1.102323e+06,891.4861,34.695014,0.003011,-0.003841
2,79.211923,334.851724,397985.547970,-1.102324e+06,891.3228,69.371314,0.003946,-0.003423
3,79.211808,334.853304,398020.212712,-1.102325e+06,891.1725,104.046285,0.004970,-0.002393
4,79.211692,334.854884,398054.914911,-1.102326e+06,891.0138,138.761317,0.005104,-0.003135


## Convert to GeoDataFrame

In [244]:
#create new geometry column:
final_data['geometry'] = list(zip(final_data['PS_x'], final_data['PS_y']))

#create new shapely Point geometry objects 
final_data['geometry'] = final_data['geometry'].apply(Point)
#final_data.head()

#convert DataFrame to GeoDataFrame in Polar Stereo
final_gdf = gpd.GeoDataFrame(final_data, crs={'init' :'epsg:3413'})

final_gdf.head()

,ATM_lat,ATM_long,PS_x,PS_y,ATM_elev,dist_along,slope_NS,slope_EW,geometry
0,79.212155,334.848567,397916.203061,-1.102322e+06,891.6397,0.000000,0.002528,-0.003607,POINT (397916.2030612105 -1102322.114924406)
1,79.212039,334.850146,397950.884957,-1.102323e+06,891.4861,34.695014,0.003011,-0.003841,POINT (397950.8849572688 -1102323.068921348)
2,79.211923,334.851724,397985.547970,-1.102324e+06,891.3228,69.371314,0.003946,-0.003423,POINT (397985.547969724 -1102324.028792245)
3,79.211808,334.853304,398020.212712,-1.102325e+06,891.1725,104.046285,0.004970,-0.002393,POINT (398020.2127118901 -1102324.870937772)
4,79.211692,334.854884,398054.914911,-1.102326e+06,891.0138,138.761317,0.005104,-0.003135,POINT (398054.9149109663 -1102325.814771683)


## Plotting the data:

In [268]:
#2D elevation cross-section plot with DataFrame:
elev_prof = final_data.plot(x='dist_along', y='ATM_elev', kind='scatter', s=0.5)
elev_prof.set_xlabel("Distance along glacier flowline (m)")
elev_prof.set_ylabel("WGS84 Ellipsoid Height (m)")

#2D plot of track (X, Y, by elevation) with DataFrame:
# track = final_data.plot(x='PS_x', y='PS_y', kind='scatter', s=0.05, c='ATM_elev', cmap='inferno')
# track.set_xlabel("Polar Stereographic X (m)")
# track.set_ylabel("Polar Stereographic Y (m)")

#plot with GeoDataFrame
# gdf_plot = final_gdf.plot('ATM_elev', s=0.05, cmap='inferno', legend=True);
# gdf_plot.set_xlabel("Polar Stereographic X (m)")
# gdf_plot.set_ylabel("Polar Stereographic Y (m)")

/srv/conda/lib/python3.6/site-packages/pandas/plotting/_core.py:304: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = self.plt.figure(figsize=self.figsize)


FigureCanvasNbAgg()

Text(0, 0.5, 'WGS84 Ellipsoid Height (m)')

## Playing around with slope:

In [269]:
elev_prof = final_data.plot(x='dist_along', y='ATM_elev', kind='scatter', s=0.5, c='slope_NS', cmap='inferno')
elev_prof.set_xlabel("Distance along glacier flowline (m)")
elev_prof.set_ylabel("WGS84 Ellipsoid Height (m)")

# final_data.hist('slope_NS', bins=50)

/srv/conda/lib/python3.6/site-packages/pandas/plotting/_core.py:304: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = self.plt.figure(figsize=self.figsize)


FigureCanvasNbAgg()

Text(0, 0.5, 'WGS84 Ellipsoid Height (m)')